Takes in .AWD, sleep log if it exists, calendar, and spits out some pretty pictures

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, '/data/MoodGroup/actigraphy/gavi/actigraPy') 

import actigraPy.actigraPy as act
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

/data/MoodGroup/actigraphy/gavi/actigraPy/actigraPy/actigraPy.py:23: MatplotlibDeprecationWarning: 
The matplotlib.backends.tkagg module was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  import matplotlib.backends.tkagg as tkagg


In [2]:
# data and log directories
sub = 'TEST' #subject NUMBER as STRING
sub_long = 'TEST'

out_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data/output' #output directory
data_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


## Subject and directory info

In [3]:
fn = {}
#.AWD
awd_fn = os.path.join(data_dir, sub_long+'.AWD')
#sleeplog
fn['cal']= os.path.join(data_dir, sub_long + '_calendar_log.xls')
#calendar
fn['log'] = os.path.join(data_dir, sub_long+'_sleeplog.xls')
logs = []
Mtimes_fn = os.path.join(out_dir,sub_long+'_Mtimes_edit.csv')

if os.path.isfile(awd_fn):
    for name in fn.keys():
        if os.path.isfile(fn[name]):
            print("%s has %s file at %s"%(sub,name,fn[name]))
            logs.append(fn[name])
        else:
            print("no " + name)
            fn[name] = ''
else:
    print("DO NOT CONTINUE!! THERE IS NO AWD FILE")

TEST has cal file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_calendar_log.xls
TEST has log file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_sleeplog.xls


### Read Mtimes file

Figure out where to clip awd file (for plotting purposes)

In [4]:
#read AWD file
awd_dat = act.read_AWD(awd_fn)
idx={'start':[0],'end':[len(awd_dat['dt_list'])-1]}
#get the start and stops from each log if they exist
for log in logs:
    _, kw_dat= act.read_log(log,awd_dat)
    
    if 'watch_on' in kw_dat.keys():
        on_date = kw_dat['watch_on'].iloc[0]['OnDate']
        on_time = kw_dat['watch_on'].iloc[0]['OnTime']
        on = datetime(on_date.year,on_date.month,on_date.day,on_time.hour,on_time.minute)
        #check if that time is in dt_list:
        if awd_dat['dt_list'].count(on) > 0:
            on_idx=awd_dat['dt_list'].index(on)
            idx['start'].append(on_idx)

    if 'watch_off' in kw_dat.keys():
        off_date = kw_dat['watch_off'].iloc[0]['OffDate']
        off_time = kw_dat['watch_off'].iloc[0]['OffTime']
        off = datetime(off_date.year,off_date.month,off_date.day,off_time.hour,off_time.minute)
        if awd_dat['dt_list'].count(off) > 0:
            off_idx=awd_dat['dt_list'].index(off)
            idx['end'].append(off_idx)
 
idx

     OffDate   OffTime     OnDate    OnTime                Comment
0        NaT       NaN 2016-02-08  10:00:00                  start
1 2016-02-07  13:00:00 2016-02-07  14:30:00  Fake, for bug testing
2 2016-02-08  13:00:00 2016-02-08  14:30:00                     3T
3 2016-02-08  21:30:00 2016-02-09  07:00:00            Sleep study
4 2016-02-09  14:00:00 2016-02-09  16:00:00                     7T
5 2016-02-10  10:00:00 2016-02-10  12:00:00                    NaN
6 2016-02-10  21:30:00 2016-02-11  07:00:00            Sleep study
7 2016-02-11  09:00:00 2016-02-11  09:40:00               Infusion
8 2016-02-11  15:00:00 2016-02-11  17:00:00                    MEG
     OffDate   OffTime     OnDate    OnTime Comment
0        NaT       NaN 2016-02-08  15:40:00   start
1 2016-02-09  00:42:00 2016-02-09  01:20:00  shower
2 2016-02-09  13:45:00 2016-02-09  16:35:00  7T MRI
3 2016-02-09  23:25:00 2016-02-10  00:05:00  shower
4 2016-02-10  19:15:00 2016-02-10  20:07:00  shower
5 2016-02-11  15:0

{'start': [0, 144], 'end': [4741]}

## <font color='red'>This part needs user input! Use your judgment to decide which starts and ends from above to go with</font>


In [5]:
start = max(idx['start'])
end = min(idx['end'])
print('start = %d, end = %d'%(start,end))

start = 144, end = 4741


In [6]:
#modify these according to output from above!!
#start =0
#end = 45283
lim_tup = [(awd_dat['DateTime'][start],awd_dat['DateTime'][end],"")]
clip_dat = act.clip_dat(lim,awd_dat)

NameError: name 'lim' is not defined

## Now just run these cells in order to extract comments and write dat/JSON

In [ ]:
importlib.reload(act)
log_dat,kw_dat=act.read_log(Mtimes_fn,awd_dat)

In [ ]:
importlib.reload(act)
dat = act.write_dat(awd_dat,mk_idx,os.path.join(out_dir,sub_long))

## Make graphs

In [ ]:
wanted_plots=['g']
plots = {}
for name in wanted_plots:
    if name in mk_idx.keys():
        plots[name]=mk_idx[name]

In [ ]:
importlib.reload(act)

act.plot_awd(awd_dat,plots,max_act=500,show=False,fn_pref=os.path.join(out_dir,sub+'_M+edits'),plot_type='single',debug=True)
